<a href="https://colab.research.google.com/github/tiagosilveiraa/portifoliods/blob/main/Recomenda%C3%A7%C3%A3o_de_Jogadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [294]:
#@title
import numpy as np
import pandas as pd
import requests
from datetime import date

In [295]:
#@title
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)

In [296]:
#@title
leagues = requests.get('https://api.sofascore.com/api/v1/config/unique-tournaments/BR/football').json()['uniqueTournaments']
seasons = []

In [297]:
#@title
dp_leagues_data = []
dp_seasons_data = []
for l in leagues:  
  dp_leagues_data.append((l['name'], l['id']))
  s = requests.get(f'https://api.sofascore.com/api/v1/unique-tournament/{l["id"]}/seasons').json()['seasons']  
  for si in s:
    si['league'] = l['id']
    dp_seasons_data.append((si['year'], si['id'], si['league']))      

#1º Parametrização



In [298]:
#@title No menu abaixo selecione as ligas e temporadas que deseja obter os dados.
import ipywidgets as widgets
from IPython.display import display
import time
add_button = widgets.Button(description="Adicionar")
clear_button = widgets.Button(description="Limpar Seleções")
dp_leagues = widgets.Dropdown(options=dp_leagues_data, value=None)
dp_seasons = widgets.Dropdown(value=None)
output = widgets.Output()
selected_leagues = []
selected_seasons = []

def on_change(event):
  dp_seasons_league = []
  if event['name'] == 'value':
    for ds in dp_seasons_data:
      #Busco apenas pelas seasons da liga selecionada pela posição na tupla
      if ds[2] == event['new']: 
        dp_seasons_league.append(ds[0:2])
    dp_seasons.options = dp_seasons_league

def add_league(b):
  if dp_leagues.value == None or dp_seasons.value == None:
    output.clear_output()
    print('Informações não preenchidas!')
  elif dp_leagues.value in selected_leagues and dp_seasons.value in selected_seasons:
    with output:      
      output.clear_output()
      print(f"Erro: Liga e temporada já adicionadas")       
  else:
    selected_leagues.append(dp_leagues.value)
    selected_seasons.append(dp_seasons.value) 
    output.clear_output()   
    with output:      
      print('Liga adicionada com sucesso! :D')

def clear_selection(b):
  selected_leagues.clear()
  selected_seasons.clear()
  with output:      
      print('Parâmetros reiniciados.')
      
dp_leagues.observe(on_change)
add_button.on_click(add_league)
clear_button.on_click(clear_selection)
display(dp_leagues, dp_seasons, add_button, clear_button,   output)

Dropdown(options=(('European Championship', 1), ('Copa América', 133), ('World Cup', 16), ('UEFA Champions Lea…

Dropdown(options=(), value=None)

Button(description='Adicionar', style=ButtonStyle())

Button(description='Limpar Seleções', style=ButtonStyle())

Output()

Agora selecione as posições dos jogadores que terão seus dados extraídos.

*Para selecionar mais de uma posição use as teclas CRTL ou SHIFT*

In [299]:
#@title
dp_positions = widgets.SelectMultiple(
    options=[('Goleiros', 'G'), ('Defensores', 'D'), ('Meio-Campistas', 'M'), ('Atacantes','F')],    
    value=['G', 'D', 'M', 'F'],
    #rows=10,
    #description='Posições',
    disabled=False
)
display(dp_positions)

SelectMultiple(index=(0, 1, 2, 3), options=(('Goleiros', 'G'), ('Defensores', 'D'), ('Meio-Campistas', 'M'), (…

#2º Obtenção das estatisticas
Nesta etapa iremos utilizar os parâmetros definidos acima para capturar os dados no SofaScore

In [ ]:
#@title
def make_url_stats(competition, season, position, offset):
  return f"https://api.sofascore.com/api/v1/unique-tournament/{competition}/season/{season}/statistics?offset={offset}&order=-rating&accumulation=total&filters=position.in.{position}&fields=%2CbigChancesMissed%2CsuccessfulDribbles%2CsuccessfulDribblesPercentage%2CtotalShots%2CshotsOnTarget%2CshotsOffTarget%2CblockedShots%2CgoalConversionPercentage%2CpenaltiesTaken%2CpenaltyGoals%2CpenaltyWon%2CshotFromSetPiece%2CfreeKickGoal%2CgoalsFromInsideTheBox%2CgoalsFromOutsideTheBox%2CheadedGoals%2CleftFootGoals%2CrightFootGoal%2CshitWoodwork%2Coffsides%2CpenaltyConversion%2CsetPieceConversion%2Crating%2Cinterceptions%2CpenaltyConceded%2Cclearances%2CerrorLeadToGoal%2CerrorLeadToShot%2CownGoals%2CdribbledPast%2CcleanSheet%2C rating %2CbigChancesCreated%2Cassists %2CaccuratePasses%2CinaccuratePasses%2CtotalPasses%2CaccuratePassesPercentage %2CaccurateOwnHalfPasses%2CaccurateOppositionHalfPasses%2CaccurateFinalThirdPasses%2CkeyPasses%2CaccurateCrosses%2CaccurateCrossesPercentage%2CaccurateLongBalls%2CaccurateLongBallsPercentage%2CpassToAssist%2Crating %2CcleanSheet%2CpenaltyFaced%2CpenaltySave%2CsavedShotsFromInsideTheBox%2CsavedShotsFromOutsideTheBox%2CgoalsConcededInsideTheBox%2CgoalsConcededOutsideTheBox%2Cpunches%2CsuccessfulRunsOut%2ChighClaims%2CcrossesNotClaimed%2Crating %2CyellowCards%2CredCards%2CgroundDuelsWon%2CgroundDuelsWonPercentage%2CaerialDuelsWon%2CaerialDuelsWonPercentage%2CtotalDuelsWon%2CtotalDuelsWonPercentage%2CminutesPlayed%2CwasFouled%2Cfouls%2Cdispossessed%2CpossessionLost%2Cappearances%2CmatchesStarted%2Crating &limit=20"

In [ ]:
#@title
def get_stats(leagues, seasons, positions):
  stats = pd.DataFrame()  
  for idx, l in enumerate(leagues):    
    for p in positions:
     iw = 0
     while True:  
       response = requests.get(make_url_stats(l, seasons[idx], p, iw))
       #Já percoreu todos os players então paro
       if response.json()['results'] == []:        
         break;

       response = response.json()['results']
       response = pd.json_normalize(response, max_level=6)
       response['position'] = p        
       stats = pd.concat([stats, response])    
       iw = iw + 20 
  return stats

In [434]:
#@title
players = get_stats(selected_leagues, selected_seasons, dp_positions.value)

#3º - Enriquecimento dos dados
Na etapa anterior obtivemos todas as estatisticas disponíveis para os jogadores, agora vamos fazer o enriquecimento desses dados com outras informações.

In [416]:
#TODO: Melhorar a velocidade
extras = pd.DataFrame()
for p in players['player.id']:
  attributes = requests.get(f'https://api.sofascore.com/api/v1/player/{p}').json()['player']
  alternative_positions = requests.get(f'https://api.sofascore.com/api/v1/player/{p}/characteristics').json()['positions']
  attributes['alternative_positions'] = (','.join(alternative_positions) if len(alternative_positions) > 1 else 'UNDEFINED')
  attributes['player.id'] = p
  attributes = pd.json_normalize(attributes, max_level=4)
  extras = pd.concat([extras, attributes])

  

In [427]:
#Como vieram muitas informações redundantes iremos dropá-las
extras = extras.drop(columns=['name', 'slug', 'shortName', 'position', 'jerseyNumber','userCount', 'id','team.name', 'team.slug',
       'team.shortName', 'team.gender', 'team.sport.name', 'team.sport.slug',
       'team.sport.id', 'team.tournament.name', 'team.tournament.slug',
       'team.tournament.category.name', 'team.tournament.category.slug',
       'team.tournament.category.sport.name',
       'team.tournament.category.sport.slug',
       'team.tournament.category.sport.id', 'team.tournament.category.id',
       'team.tournament.category.flag', 'team.tournament.category.alpha2',
       'team.tournament.uniqueTournament.name',
       'team.tournament.uniqueTournament.slug',
       'team.tournament.uniqueTournament.category.name',
       'team.tournament.uniqueTournament.category.slug',
       'team.tournament.uniqueTournament.category.sport',
       'team.tournament.uniqueTournament.category.id',
       'team.tournament.uniqueTournament.category.flag',
       'team.tournament.uniqueTournament.category.alpha2',
       'team.tournament.uniqueTournament.userCount',
       'team.tournament.uniqueTournament.id',
       'team.tournament.uniqueTournament.displayInverseHomeAwayTeams',
       'team.tournament.priority', 'team.tournament.id',
       'team.primaryUniqueTournament.name',
       'team.primaryUniqueTournament.slug',
       'team.primaryUniqueTournament.category.name',
       'team.primaryUniqueTournament.category.slug',
       'team.primaryUniqueTournament.category.sport.name',
       'team.primaryUniqueTournament.category.sport.slug',
       'team.primaryUniqueTournament.category.sport.id',
       'team.primaryUniqueTournament.category.id',
       'team.primaryUniqueTournament.category.flag',
       'team.primaryUniqueTournament.category.alpha2',
       'team.primaryUniqueTournament.userCount',
       'team.primaryUniqueTournament.id',
       'team.primaryUniqueTournament.displayInverseHomeAwayTeams',
       'team.userCount', 'team.nameCode', 'team.disabled', 'team.national',
       'team.type', 'team.id', 'team.country.alpha2', 'team.country.name',
       'team.teamColors.primary', 'team.teamColors.secondary',
       'team.teamColors.text', 'proposedMarketValue'])

In [433]:
extras.rename(columns={'country.alpha2':'countryCode', 'country.name':'countryName', 'proposedMarketValueRaw.value':'proposedMarketValue', 'proposedMarketValueRaw.currency': 'proposedMarketCurrency'}, inplace=True)

In [435]:
players = players.merge(extras, on='player.id')

In [436]:
players

,bigChancesMissed,successfulDribbles,successfulDribblesPercentage,totalShots,shotsOnTarget,shotsOffTarget,blockedShots,goalConversionPercentage,penaltiesTaken,penaltyGoals,penaltyWon,shotFromSetPiece,freeKickGoal,goalsFromInsideTheBox,goalsFromOutsideTheBox,headedGoals,leftFootGoals,offsides,penaltyConversion,setPieceConversion,rating,interceptions,penaltyConceded,clearances,errorLeadToGoal,errorLeadToShot,ownGoals,dribbledPast,cleanSheet,bigChancesCreated,accuratePasses,inaccuratePasses,totalPasses,accurateOwnHalfPasses,accurateOppositionHalfPasses,accurateFinalThirdPasses,keyPasses,accurateCrosses,accurateCrossesPercentage,accurateLongBalls,accurateLongBallsPercentage,passToAssist,penaltyFaced,penaltySave,savedShotsFromInsideTheBox,savedShotsFromOutsideTheBox,goalsConcededInsideTheBox,goalsConcededOutsideTheBox,punches,successfulRunsOut,highClaims,crossesNotClaimed,yellowCards,redCards,groundDuelsWon,groundDuelsWonPercentage,aerialDuelsWon,aerialDuelsWonPercentage,totalDuelsWon,totalDuelsWonPercentage,minutesPlayed,wasFouled,fouls,dispossessed,possessionLost,appearances,matchesStarted,player.name,player.slug,player.userCount,player.id,team.name,team.slug,team.shortName,team.sport.name,team.sport.slug,team.sport.id,team.userCount,team.type,team.id,team.teamColors.primary,team.teamColors.secondary,team.teamColors.text,position,height,preferredFoot,shirtNumber,dateOfBirthTimestamp,contractUntilTimestamp,alternative_positions,countryCode,countryName,proposedMarketValue,proposedMarketCurrency,firstName,lastName,retired
0,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.95,0,0,7,0,0,0,0,1,0,29,16,45,21,8,1,0,0,0.0,11,42.31,0,0,0,5,5,1,0,4,0,2,0,1,0,1,100.00,2,100.0,3,100.00,180,1,0,0,16,2,2,Renan,renan,49,39999,Atlético Goianiense,atletico-goianiense,Atlético Goianiense,Football,football,1,0,0,7314,#52b030,#52b030,#ffffff,G,189.0,Right,1.0,6.114528e+08,1.672445e+09,UNDEFINED,BR,Brazil,370000.0,EUR,NaN,NaN,NaN
1,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.90,0,0,1,0,0,0,0,0,0,11,10,21,10,1,0,0,0,0.0,4,28.57,0,0,0,5,2,1,0,1,0,1,0,0,0,0,0.00,0,0.0,0,0.00,90,0,0,0,12,1,1,Airton,airton,42,797866,América Mineiro,america-mineiro,América Mineiro,Football,football,1,0,0,1973,#52b030,#52b030,#ffffff,G,188.0,Right,12.0,7.701696e+08,1.672445e+09,UNDEFINED,BR,Brazil,260000.0,EUR,,,NaN
2,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.70,0,0,2,0,0,0,0,2,0,53,6,59,46,7,0,0,0,0.0,11,64.71,0,0,0,3,4,0,0,1,1,1,0,0,0,0,0.00,1,100.0,1,100.00,180,1,0,0,6,2,2,Marcelo Lomba,marcelo-lomba,358,34141,Palmeiras,palmeiras,Palmeiras,Football,football,1,0,0,1963,#52b030,#52b030,#ffffff,G,187.0,Right,42.0,5.352480e+08,1.672445e+09,UNDEFINED,BR,Brazil,810000.0,EUR,,,NaN
3,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.54,0,0,2,0,0,0,0,5,0,62,53,115,33,29,8,0,0,0.0,34,39.08,0,0,0,11,7,0,0,2,0,3,0,1,0,0,0.00,0,0.0,0,0.00,450,0,0,0,54,5,5,Fernando Miguel,fernando-miguel,210,358418,Fortaleza,fortaleza,Fortaleza,Football,football,1,0,0,2020,#52b030,#52b030,#ffffff,G,191.0,Left,16.0,4.761504e+08,1.672445e+09,UNDEFINED,BR,Brazil,505000.0,EUR,NaN,NaN,NaN
4,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.30,0,0,0,0,0,0,0,0,0,9,13,22,8,1,0,0,0,0.0,2,13.33,0,0,0,1,4,0,1,0,0,1,0,0,0,0,0.00,0,0.0,0,0.00,90,0,0,0,14,1,1,Carlos Miguel,carlos-miguel,179,973738,Corinthians,corinthians,Corinthians,Football,football,1,0,0,1957,#52b030,#52b030,#ffffff,G,204.0,Right,22.0,9.078912e+08,1.703981e+09,UNDEFINED,BR,Brazil,330000.0,EUR,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,3,1,50.0,4,0,3,1,0.0,0,0,0,0,0,0,0,0,0,2,0.0,0.0,6.36,0,0,3,0,0,0,0,0,0,15,4,19,6,9,6,4,0,0.0,0,0.00,0,0,0,0,0,2,0,0,0,0,0,1,0,6,42.86,6,60.0,

#2º Tratamento dos dados


In [437]:
#colocando todas as colunas em minusculo
players = players.rename(str.lower,axis='columns')    

In [438]:
#Só quero players que estiveram em alguma partida na temporada
players = players[players['appearances'] >0]

In [439]:
#Alterando o indice para o player.id
players.set_index('player.id', inplace = True)

In [440]:
players

,bigchancesmissed,successfuldribbles,successfuldribblespercentage,totalshots,shotsontarget,shotsofftarget,blockedshots,goalconversionpercentage,penaltiestaken,penaltygoals,penaltywon,shotfromsetpiece,freekickgoal,goalsfrominsidethebox,goalsfromoutsidethebox,headedgoals,leftfootgoals,offsides,penaltyconversion,setpiececonversion,rating,interceptions,penaltyconceded,clearances,errorleadtogoal,errorleadtoshot,owngoals,dribbledpast,cleansheet,bigchancescreated,accuratepasses,inaccuratepasses,totalpasses,accurateownhalfpasses,accurateoppositionhalfpasses,accuratefinalthirdpasses,keypasses,accuratecrosses,accuratecrossespercentage,accuratelongballs,accuratelongballspercentage,passtoassist,penaltyfaced,penaltysave,savedshotsfrominsidethebox,savedshotsfromoutsidethebox,goalsconcededinsidethebox,goalsconcededoutsidethebox,punches,successfulrunsout,highclaims,crossesnotclaimed,yellowcards,redcards,groundduelswon,groundduelswonpercentage,aerialduelswon,aerialduelswonpercentage,totalduelswon,totalduelswonpercentage,minutesplayed,wasfouled,fouls,dispossessed,possessionlost,appearances,matchesstarted,player.name,player.slug,player.usercount,team.name,team.slug,team.shortname,team.sport.name,team.sport.slug,team.sport.id,team.usercount,team.type,team.id,team.teamcolors.primary,team.teamcolors.secondary,team.teamcolors.text,position,height,preferredfoot,shirtnumber,dateofbirthtimestamp,contractuntiltimestamp,alternative_positions,countrycode,countryname,proposedmarketvalue,proposedmarketcurrency,firstname,lastname,retired
player.id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
39999,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.95,0,0,7,0,0,0,0,1,0,29,16,45,21,8,1,0,0,0.0,11,42.31,0,0,0,5,5,1,0,4,0,2,0,1,0,1,100.00,2,100.00,3,100.00,180,1,0,0,16,2,2,Renan,renan,49,Atlético Goianiense,atletico-goianiense,Atlético Goianiense,Football,football,1,0,0,7314,#52b030,#52b030,#ffffff,G,189.0,Right,1.0,6.114528e+08,1.672445e+09,UNDEFINED,BR,Brazil,370000.0,EUR,NaN,NaN,NaN
797866,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.90,0,0,1,0,0,0,0,0,0,11,10,21,10,1,0,0,0,0.0,4,28.57,0,0,0,5,2,1,0,1,0,1,0,0,0,0,0.00,0,0.00,0,0.00,90,0,0,0,12,1,1,Airton,airton,42,América Mineiro,america-mineiro,América Mineiro,Football,football,1,0,0,1973,#52b030,#52b030,#ffffff,G,188.0,Right,12.0,7.701696e+08,1.672445e+09,UNDEFINED,BR,Brazil,260000.0,EUR,,,NaN
34141,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.70,0,0,2,0,0,0,0,2,0,53,6,59,46,7,0,0,0,0.0,11,64.71,0,0,0,3,4,0,0,1,1,1,0,0,0,0,0.00,1,100.00,1,100.00,180,1,0,0,6,2,2,Marcelo Lomba,marcelo-lomba,358,Palmeiras,palmeiras,Palmeiras,Football,football,1,0,0,1963,#52b030,#52b030,#ffffff,G,187.0,Right,42.0,5.352480e+08,1.672445e+09,UNDEFINED,BR,Brazil,810000.0,EUR,,,NaN
358418,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.54,0,0,2,0,0,0,0,5,0,62,53,115,33,29,8,0,0,0.0,34,39.08,0,0,0,11,7,0,0,2,0,3,0,1,0,0,0.00,0,0.00,0,0.00,450,0,0,0,54,5,5,Fernando Miguel,fernando-miguel,210,Fortaleza,fortaleza,Fortaleza,Football,football,1,0,0,2020,#52b030,#52b030,#ffffff,G,191.0,Left,16.0,4.761504e+08,1.672445e+09,UNDEFINED,BR,Brazil,505000.0,EUR,NaN,NaN,NaN
973738,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.30,0,0,0,0,0,0,0,0,0,9,13,22,8,1,0,0,0,0.0,2,13.33,0,0,0,1,4,0,1,0,0,1,0,0,0,0,0.00,0,0.00,0,0.00,90,0,0,0,14,1,1,Carlos Miguel,carlos-miguel,179,Corinthians,corinthians,Corinthians,Football,football,1,0,0,1957,#52b030,#52b030,#ffffff,G,204.0,Right,22.0,9.078912e+08,1.703981e+09,UNDEFINED,BR,Brazil,330000.0,EUR,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790103,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,6.38

#3º Análise exploratória



#4° Geração do Modelo

In [ ]:
numeric_stats = players.drop([
    'team.name',
'player.name',
'player.slug',
'player.usercount',
'team.name',
'team.slug',
'team.shortname',
'team.sport.name',
'team.sport.slug',
'team.sport.id',
'team.usercount',
'team.type',
'team.id',
'team.teamcolors.primary',
'team.teamcolors.secondary',
'team.teamcolors.text',
'position',
'rating'
], axis='columns')

In [ ]:
numeric_stats.columns

In [ ]:
from scipy.sparse import csr_matrix
stats_sparse = csr_matrix(numeric_stats)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute', n_neighbors=5)
model.fit(stats_sparse)

In [ ]:
distance, similar_players_index = model.kneighbors(numeric_stats.loc[840219].values.reshape(1,-1))
similar_players = pd.DataFrame()
similar_players = players.loc[players.index[similar_players_index[0]]]
similar_players['distances'] = distance[0]

In [ ]:
similar_players